In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import brier_score_loss
%matplotlib inline
import numpy as np

In [37]:
# Load dataset
#CIDR alcohol + gambling (PRS for gambling)
#this is for nonafricans
data = pd.read_table("//cbio//projects//018//data//prs//cidr-nonafricans//Joined.txt", delim_whitespace=True)
data.loc[data['Gambling'] == 0]
data.drop(data.loc[data['Gambling'] == 0].index, inplace=True)
data.loc[data['Gambling'] == 0]
data

,IID,Sex,Age,alcdep,Gambling,Zero,Yes,PRS
0,Yale_0089,1,43,2,1,0,Yes,-0.076246
1,Yale_0109,1,22,1,1,0,Yes,-0.952154
2,Yale_0119,2,20,1,1,0,Yes,-0.762640
3,Yale_0181,1,25,1,1,0,Yes,-0.645262
4,Yale_0203,2,22,1,1,0,Yes,-1.005681
...,...,...,...,...,...,...,...,...
1039,Yale_3242,2,33,2,1,0,Yes,-0.423614
1040,Yale_3250,2,27,1,1,0,Yes,-0.087180
1041,Yale_3259,2,35,1,1,0,Yes,-0.879843
1042,Yale_3268,2,39,2,1,0,Yes,-0.366853


In [22]:
#drop the index
data=data.drop("IID", axis =1 )
data

,Sex,Age,alcdep,Gambling,Zero,Yes,PRS
0,1,43,2,1,0,Yes,-0.076246
1,1,22,1,1,0,Yes,-0.952154
2,2,20,1,1,0,Yes,-0.762640
3,1,25,1,1,0,Yes,-0.645262
4,2,22,1,1,0,Yes,-1.005681
...,...,...,...,...,...,...,...
1039,2,33,2,1,0,Yes,-0.423614
1040,2,27,1,1,0,Yes,-0.087180
1041,2,35,1,1,0,Yes,-0.879843
1042,2,39,2,1,0,Yes,-0.366853


In [23]:
#setting up the y and the x values 
#y is the acldep

X = data.drop("alcdep", axis=1)
x=X.drop("Zero", axis =1 )
x=x.drop("Yes", axis = 1)
y = data["alcdep"]
y

0       2
1       1
2       1
3       1
4       1
       ..
1039    2
1040    1
1041    1
1042    2
1043    2
Name: alcdep, Length: 1033, dtype: int64

In [38]:
data.loc[data['alcdep'] == 2]

,IID,Sex,Age,alcdep,Gambling,Zero,Yes,PRS
0,Yale_0089,1,43,2,1,0,Yes,-0.076246
12,Yale_0564,1,50,2,1,0,Yes,-0.507887
13,Yale_0587,1,25,2,1,0,Yes,-0.464685
14,Yale_0619,2,18,2,1,0,Yes,-0.464331
15,Yale_0624,1,27,2,1,0,Yes,-0.155007
...,...,...,...,...,...,...,...,...
1037,Yale_3211,1,48,2,1,0,Yes,-0.618663
1038,Yale_3236,1,37,2,1,0,Yes,-0.178210
1039,Yale_3242,2,33,2,1,0,Yes,-0.423614
1042,Yale_3268,2,39,2,1,0,Yes,-0.366853


In [39]:
#create the training and the testing dataset
np.random.seed(42)

# Split into train & test set
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [40]:
#look at the alcdep for nonafricans
y_train

659    2
361    2
533    2
451    2
993    2
      ..
88     2
336    2
472    2
124    2
870    2
Name: alcdep, Length: 826, dtype: int64

In [41]:
#create the model
lr = LogisticRegression(solver='liblinear')
lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [42]:
#IDK why this is here
probs = lr.predict_proba(X_test)
probs = probs[:, 1] # Keeping only the values in positive label

In [43]:
#this either
np.mean(probs)

0.013917900750273639

In [44]:
#count how many were predicted to have alcohol dependence
y_pred1=lr.predict(X_test)
count = np.count_nonzero(y_pred1)
count

207

In [45]:
#2 is affected by gambling addiction and alcohol addiction

y_pred1=pd.DataFrame(y_pred1, columns = ["alcdep_predicted"])
data5=y_pred1.loc[(y_pred1['alcdep_predicted']==2)]
data5

,alcdep_predicted
0,2
1,2
2,2
3,2
4,2
...,...
202,2
203,2
204,2
205,2


In [46]:
#2 is affected by gambling addiction and alcohol addiction

y_pred1=pd.DataFrame(y_pred1, columns = ["alcdep_predicted"])
data5=y_pred1.loc[(y_pred1['alcdep_predicted']==2) & (y_train==2) ]
data5

,alcdep_predicted
0,2
12,2
13,2
14,2
15,2
...,...
200,2
201,2
204,2
205,2


In [47]:
#Confusion matrix, Accuracy, sensitivity and specificity
from sklearn.metrics import confusion_matrix

cm1 = confusion_matrix(y_test,y_pred1)
print('Confusion Matrix : \n', cm1)

total1=sum(sum(cm1))
#####from confusion matrix calculate accuracy
accuracy1=(cm1[0,0]+cm1[1,1])/total1
print ('Accuracy : ', accuracy1)

sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
print('Sensitivity : ', sensitivity1 )

specificity1 = cm1[1,1]/(cm1[1,0]+cm1[1,1])
print('Specificity : ', specificity1)

Confusion Matrix : 
 [[  1   2]
 [  0 204]]
Accuracy :  0.9903381642512077
Sensitivity :  0.3333333333333333
Specificity :  1.0


In [ ]:
#Looking to see where gambling occurs

In [48]:
#count how many were predicted to have alcohol dependence
y_pred1=lr.predict(X_test)
X_test

,Sex,Age,Gambling,PRS
299,2,27,1,-0.053419
459,2,36,1,-0.046786
646,1,29,1,-0.237225
142,1,41,1,-0.047024
544,1,23,1,-0.050562
...,...,...,...,...
951,1,20,1,-0.050054
363,1,34,2,-0.043167
491,2,49,1,-0.059169
604,1,26,1,-0.051821


In [49]:
#2 is affected by gambling addiction and alcohol addiction
newdf=pd.DataFrame(X_test["Gambling"], columns = ["Gambling"])
y_pred1=pd.DataFrame(y_pred1, columns = ["alcdep_predicted"])
data5=y_pred1.loc[(y_pred1['alcdep_predicted']==2) & (y_test==2) & (newdf["Gambling"]==2)]
data5

,alcdep_predicted
108,2


In [50]:
data6=data.loc[(data['alcdep']==2) & (data["Gambling"]==2)]
data6

,IID,Sex,Age,alcdep,Gambling,Zero,Yes,PRS
18,Yale_0700,2,22,2,2,0,Yes,-0.647318
33,Yale_1075,2,37,2,2,0,Yes,-0.363624
43,Yale_1223,1,18,2,2,0,Yes,-0.337125
57,Yale_1505,2,39,2,2,0,Yes,-0.659855
69,Yale_1796,2,45,2,2,0,Yes,-0.562967
...,...,...,...,...,...,...,...,...
995,Yale_3044,1,57,2,2,0,Yes,-0.048670
1002,Yale_3051,1,50,2,2,0,Yes,-0.051788
1015,Yale_3066,1,53,2,2,0,Yes,-0.053495
1026,Yale_3080,1,45,2,2,0,Yes,-0.164243


In [22]:
# Load dataset
#CIDR alcohol + gambling (PRS for gambling)
data2 = pd.read_table("//cbio//projects//018//data//prs//cidr-africans//Joined.txt", delim_whitespace=True)
data2.loc[data2['Gambling'] == 0]
data2.drop(data2.loc[data2['Gambling'] == 0].index, inplace=True)
data2.loc[data2['Gambling'] == 0]

,IID,Sex,Age,alcdep,Gambling,Zero,Yes,PRS


In [23]:
data2=data2.drop("IID", axis =1 )
data2

,Sex,Age,alcdep,Gambling,Zero,Yes,PRS
0,2,27,2,1,0,Yes,0.004705
1,1,63,2,1,0,Yes,0.001017
2,1,39,2,1,0,Yes,0.002701
3,1,40,2,1,0,Yes,0.001085
4,2,41,2,1,0,Yes,0.001622
...,...,...,...,...,...,...,...
1840,1,36,1,1,0,Yes,0.000763
1841,2,27,1,1,0,Yes,-0.001052
1842,2,38,1,1,0,Yes,-0.000757
1843,2,26,1,1,0,Yes,-0.000448


In [24]:
X = data2.drop("Gambling", axis=1)
x=X.drop("Zero", axis =1 )
x=x.drop("Yes", axis = 1)
y = data2["Gambling"]

In [25]:
np.random.seed(42)

# Split into train & test set
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [26]:
lr2 = LogisticRegression()
lr2.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [27]:
probs = lr2.predict_proba(X_test)
probs = probs[:, 1] # Keeping only the values in positive label

In [28]:
np.mean(probs)

0.10387059099401484

In [29]:
y_pred1=lr2.predict(X_test)
y_pred = (lr2.predict_proba(X_test)[:,1] >= 0.09813021401964486).astype(bool) # set threshold as 0.251
count = np.count_nonzero(y_pred)
count

249

In [30]:
#Confusion matrix, Accuracy, sensitivity and specificity
from sklearn.metrics import confusion_matrix

cm1 = confusion_matrix(y_test,y_pred1)
print('Confusion Matrix : \n', cm1)

total1=sum(sum(cm1))
#####from confusion matrix calculate accuracy
accuracy1=(cm1[0,0]+cm1[1,1])/total1
print ('Accuracy : ', accuracy1)

sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
print('Sensitivity : ', sensitivity1 )

specificity1 = cm1[1,1]/(cm1[1,0]+cm1[1,1])
print('Specificity : ', specificity1)

Confusion Matrix : 
 [[495   0]
 [ 54   0]]
Accuracy :  0.9016393442622951
Sensitivity :  1.0
Specificity :  0.0
